## Digital Image Processing, ECE419

Ernesto Colon

The Cooper Union Department of Electrical Engineering


February 19th, 2021

References:
- Digital Image Processing Using Matlab [DIPUM] by Gonzalez et al., 3rd ed
- Digital Image Processing by Gonzalez et al., 4th ed

**The following set of notes and example scripts are part of my independent study sessions and adapted from the references outlined above. My main reference is DIPUM and complemented with online documentation. As I go through the theory and textbook examples, I am translating the code and image processing techniques to Python.**

## Image Transformations Notes/Exercises/Sandbox

In [1]:
#Importing the libraries
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import ec_img_utils
import skimage.util
from skimage import io
import scipy.fft
from skimage import transform

#### Brief Notes on Inverse Transformation / Mapping

Starting with two spatial coordinate systems called the *input space*, $(w,z)$, and the *output space*, $(x,y)$, we can
define a geometric coordinate transformation that maps the input space to the output space:

\begin{equation}
(x,y) = T[(w,z)]
\end{equation}

where $T[\cdot]$ is called the *forward transformation* or *forward mapping*. If the forward mapping has an inverse, we
can express the output space in terms of the input space points as:

\begin{equation}
(w,z) = T^{-1}[(x,y)]
\end{equation}

where $T^{-1}[\cdot]$ is called the *inverse transformation* or *inverse mapping*.

#### Affine Transformations

An affine transformation is a mapping from one vector space to another, consisting of a linear part that is expressed
as a matrix multiplication and an additive part that is an *offset* or *translation*.

Following the notation used in DIPUM 3rd edition, an affine transformation can be expressed as:



\begin{bmatrix} x & y \end{bmatrix}
= \begin{bmatrix} w & z \end{bmatrix}
\begin{bmatrix}
a_{11} & a_{12}\\
a_{21} & a_{22}
\end{bmatrix} + \begin{bmatrix} b_1 & b_2 \end{bmatrix}

We can re-write our affine transformation equation using homogeneous coordinates as follows:

\begin{bmatrix} x & y & 1 \end{bmatrix}
= \begin{bmatrix} w & z & 1 \end{bmatrix}
\begin{bmatrix}
a_{11} & a_{12} & 0\\
a_{21} & a_{22} & 0\\
b_1 & b_2 & 1
\end{bmatrix} =  \begin{bmatrix} w & z & 1 \end{bmatrix} $\textbf{T}$


where $\textbf{T}$ is called an *affine matrix* and the third column of $\textbf{T}$ is always
\begin{bmatrix} 0 & 0 & 1 \end{bmatrix}$^{t}$

#### Types of Affine Transformations

**Identity**

$\textbf{T}$ = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w$

$y = z$


**Scaling**

$\textbf{T}$ = \begin{bmatrix}
s_x & 0 & 0\\
0 & s_y & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = s_xw$

$y = s_yz$

**Rotation**

$\textbf{T}$ = \begin{bmatrix}
cos\theta & sin\theta & 0\\
-sin\theta & cos\theta & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w~cos\theta - z~sin\theta$

$y = w~sin\theta + z~cos\theta$

**Shear (horizontal)**

$\textbf{T}$ = \begin{bmatrix}
1 & 0 & 0\\
\alpha & 1 & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w + \alpha~z$

$y = z$

**Shear (vertical)**

$\textbf{T}$ = \begin{bmatrix}
1 & \beta & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w$

$y = + \beta~w + z$

**Reflection**

$\textbf{T}$ = \begin{bmatrix}
1 & 0 & 0\\
0 & -1 & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w$

$y = -z$


**Translation**

$\textbf{T}$ = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
\delta_x & \delta_y & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w + \delta_x$

$y = z + \delta_y$

**Similarity Transformation**

An affine transformation that preserves angles between lines and that changes all distances in the same ratio is called
a *similarity transformation*. In other words, a similarity transformation preserves shapes.

Examples include: rotation, translation, and reflection transformations. Scaling is also a similarity transformation if
the horizontal and vertical scale factors are the same.

We can express the affine matrix for a similarity transformation as follows:

$\textbf{T}$ = \begin{bmatrix}
s~cos\theta & s~sin\theta & 0\\
-s~sin\theta & s~cos\theta & 0\\
b_1 & b_2 & 1
\end{bmatrix}

or

$\textbf{T}$ = \begin{bmatrix}
s~cos\theta & s~sin\theta & 0\\
s~sin\theta & -s~cos\theta & 0\\
b_1 & b_2 & 1
\end{bmatrix}

where the second form is needed to account for reflections.

Similarity transformations are useful in image processing applications involving solid and flat objects. As objects
or the imaging sensor moves, we can relate the motion with similarity transformations.